In [4]:
import os
import shutil
import json
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog

# Default file categories and extensions
default_file_types = {
    "Images": [".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff"],
    "Documents": [".pdf", ".doc", ".docx", ".txt", ".xls", ".xlsx", ".ppt", ".pptx"],
    "Videos": [".mp4", ".mov", ".avi", ".mkv", ".flv"],
    "Music": [".mp3", ".wav", ".aac", ".flac"],
    "Archives": [".zip", ".rar", ".tar", ".gz", ".7z"],
    "Scripts": [".py", ".js", ".html", ".css", ".sh", ".bat"],
    "Others": []
}

class JunkFileOrganizerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Junk File Organizer")

        self.source_dir = ""
        self.log_dir = ""  # Directory to save logs and config
        self.move_logs = []  # List of move log file paths for undo history
        self.file_types = default_file_types.copy()

        # GUI Elements
        self.label = tk.Label(root, text="Select a folder to organize junk files", font=("Arial", 14))
        self.label.pack(pady=10)

        self.select_btn = tk.Button(root, text="Select Folder", command=self.select_folder)
        self.select_btn.pack(pady=5)

        self.folder_label = tk.Label(root, text="No folder selected", fg="blue")
        self.folder_label.pack(pady=5)

        self.preview_btn = tk.Button(root, text="Preview Files", command=self.preview_files, state=tk.DISABLED)
        self.preview_btn.pack(pady=5)

        self.organize_btn = tk.Button(root, text="Organize Files", command=self.organize_files, state=tk.DISABLED)
        self.organize_btn.pack(pady=5)

        self.undo_btn = tk.Button(root, text="Undo Last Organize", command=self.undo_organize, state=tk.DISABLED)
        self.undo_btn.pack(pady=5)

        self.edit_categories_btn = tk.Button(root, text="Edit File Categories", command=self.edit_categories)
        self.edit_categories_btn.pack(pady=5)

        self.status_text = tk.Text(root, height=15, width=70, state=tk.DISABLED)
        self.status_text.pack(pady=10)

        # Preview window reference
        self.preview_window = None

    def log(self, message):
        self.status_text.config(state=tk.NORMAL)
        self.status_text.insert(tk.END, message + "\n")
        self.status_text.see(tk.END)
        self.status_text.config(state=tk.DISABLED)

    def select_folder(self):
        folder = filedialog.askdirectory()
        if folder:
            self.source_dir = folder
            self.log_dir = os.path.join(self.source_dir, ".junk_organizer_logs")
            if not os.path.exists(self.log_dir):
                os.makedirs(self.log_dir)

            self.folder_label.config(text=self.source_dir)
            self.preview_btn.config(state=tk.NORMAL)
            self.organize_btn.config(state=tk.NORMAL)
            self.update_undo_button()
            self.log(f"Selected folder: {self.source_dir}")

    def update_undo_button(self):
        # Check if any move log files exist for undo
        logs = sorted(
            [f for f in os.listdir(self.log_dir) if f.startswith("file_move_log_") and f.endswith(".json")],
            reverse=True
        )
        self.move_logs = [os.path.join(self.log_dir, f) for f in logs]
        if self.move_logs:
            self.undo_btn.config(state=tk.NORMAL)
        else:
            self.undo_btn.config(state=tk.DISABLED)

    def get_category_for_extension(self, ext):
        ext = ext.lower()
        for category, extensions in self.file_types.items():
            if ext in extensions:
                return category
        return "Others"

    def preview_files(self):
        if not self.source_dir:
            messagebox.showwarning("Warning", "Please select a folder first.")
            return

        files = []
        for filename in os.listdir(self.source_dir):
            file_path = os.path.join(self.source_dir, filename)
            if os.path.isfile(file_path):
                _, ext = os.path.splitext(filename)
                category = self.get_category_for_extension(ext)
                files.append((filename, category))

        if self.preview_window and tk.Toplevel.winfo_exists(self.preview_window):
            self.preview_window.destroy()

        self.preview_window = tk.Toplevel(self.root)
        self.preview_window.title("Preview Files")

        tk.Label(self.preview_window, text="File Name", font=("Arial", 12, "bold"), width=40, anchor="w").grid(row=0, column=0, padx=5)
        tk.Label(self.preview_window, text="Category", font=("Arial", 12, "bold"), width=20, anchor="w").grid(row=0, column=1, padx=5)

        for i, (fname, cat) in enumerate(files, start=1):
            tk.Label(self.preview_window, text=fname, anchor="w", width=40).grid(row=i, column=0, sticky="w", padx=5)
            tk.Label(self.preview_window, text=cat, anchor="w", width=20).grid(row=i, column=1, sticky="w", padx=5)

        tk.Button(self.preview_window, text="Close", command=self.preview_window.destroy).grid(row=len(files)+1, column=0, columnspan=2, pady=10)

    def safe_move(self, src, dest):
        """
        Move file from src to dest.
        If dest exists, rename the file by adding a number suffix.
        """
        base, ext = os.path.splitext(dest)
        counter = 1
        new_dest = dest
        while os.path.exists(new_dest):
            new_dest = f"{base}({counter}){ext}"
            counter += 1
        shutil.move(src, new_dest)
        return new_dest

    def organize_files(self):
        if not self.source_dir:
            messagebox.showwarning("Warning", "Please select a folder first.")
            return

        # Create category folders
        for folder in self.file_types.keys():
            folder_path = os.path.join(self.source_dir, folder)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

        move_log = {}

        try:
            for filename in os.listdir(self.source_dir):
                file_path = os.path.join(self.source_dir, filename)

                if os.path.isdir(file_path):
                    continue

                _, ext = os.path.splitext(filename)
                category = self.get_category_for_extension(ext)

                dest_folder = os.path.join(self.source_dir, category)
                dest_path = os.path.join(dest_folder, filename)

                new_dest_path = self.safe_move(file_path, dest_path)
                self.log(f"Moved {filename} to {category} as {os.path.basename(new_dest_path)}")
                move_log[new_dest_path] = file_path

            # Save move log with timestamp for undo history
            import datetime
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            log_path = os.path.join(self.log_dir, f"file_move_log_{timestamp}.json")
            with open(log_path, "w") as f:
                json.dump(move_log, f, indent=4)

            self.log("Organization complete.")
            self.update_undo_button()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def undo_organize(self):
        if not self.move_logs:
            messagebox.showwarning("Warning", "No organization log found to undo.")
            return

        # Undo the most recent organize operation
        log_path = self.move_logs[0]

        try:
            with open(log_path, "r") as f:
                move_log = json.load(f)

            for new_path, original_path in move_log.items():
                if os.path.exists(new_path):
                    original_dir = os.path.dirname(original_path)
                    if not os.path.exists(original_dir):
                        os.makedirs(original_dir)
                    shutil.move(new_path, original_path)
                    self.log(f"Moved back {os.path.basename(new_path)} to original location")

            os.remove(log_path)
            self.log("Undo complete.")
            self.update_undo_button()

        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def edit_categories(self):
        # Open a new window to edit categories and extensions
        edit_win = tk.Toplevel(self.root)
        edit_win.title("Edit File Categories")

        tk.Label(edit_win, text="Category", font=("Arial", 12, "bold"), width=20).grid(row=0, column=0, padx=5, pady=5)
        tk.Label(edit_win, text="Extensions (comma separated)", font=("Arial", 12, "bold"), width=40).grid(row=0, column=1, padx=5, pady=5)

        entries = {}

        # Show current categories and extensions
        for i, (category, extensions) in enumerate(self.file_types.items(), start=1):
            cat_label = tk.Label(edit_win, text=category, width=20, anchor="w")
            cat_label.grid(row=i, column=0, sticky="w", padx=5, pady=2)

            ext_str = ", ".join(extensions)
            ext_entry = tk.Entry(edit_win, width=50)
            ext_entry.insert(0, ext_str)
            ext_entry.grid(row=i, column=1, padx=5, pady=2)
            entries[category] = ext_entry

        def save_categories():
            new_file_types = {}
            for category, entry in entries.items():
                ext_text = entry.get().strip()
                if ext_text:
                    exts = [e.strip().lower() if e.strip().startswith('.') else '.' + e.strip().lower() for e in ext_text.split(",")]
                else:
                    exts = []
                new_file_types[category] = exts

            # Ensure 'Others' category exists and is empty
            if "Others" not in new_file_types:
                new_file_types["Others"] = []

            self.file_types = new_file_types
            self.log("File categories updated.")
            edit_win.destroy()

        save_btn = tk.Button(edit_win, text="Save", command=save_categories)
        save_btn.grid(row=len(self.file_types)+1, column=0, columnspan=2, pady=10)

def main():
    root = tk.Tk()
    app = JunkFileOrganizerApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()
